In [6]:
import requests
import json
from IPython.display import JSON
from keys import *

In [7]:
#be careful not to run too often as there is a maximum of 200 api calls per month

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/LFPG/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [ ]:
# Comprueba si el código de estado HTTP de la respuesta es 200 (200 significa "OK").
if response.status_code == 200:
    new_data = response.json()

    # Inicia un bloque para manejar errores potenciales al leer el archivo.
    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)

    # Captura errores si el archivo no existe o su contenido no es JSON válido.
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []
    
    # Agrega el nuevo dato obtenido de la API a la lista de datos existentes.
    existing_data.append(new_data)
    
    # Abre el archivo "data.json" en modo escritura ("w"), lo que sobrescribe su contenido.
    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")

# Si el código de estado no fue 200, imprime un mensaje de error con el código de estado HTTP fallido
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [ ]:
# Convierte la respuesta de la API a un diccionario JSON
flight_arrivals = response.json()

# Itera sobre los vuelos en la respuesta
for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [ ]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

# Define una función que toma los datos de UN SOLO vuelo ('flights') y los formatea.
def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

# Convierte cada vuelo en un dataFrame con "flight_extraction" y luego los concatena en un único dataFrame
flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

# Muestra las primeras 5 filas del dataframe.
flight_arrivals.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,AF 7371,Lyon,Air France,Airbus A320,2025-09-01 06:00Z,2025-09-01 08:00+02:00
0,DL 262,New York,Delta Air Lines,Boeing 767-400,2025-09-01 07:05Z,2025-09-01 09:05+02:00
0,U2 2433,London,easyJet,Airbus A319,2025-09-01 06:25Z,2025-09-01 08:25+02:00
0,AF 1503,Torino,Air France,Embraer 190,2025-09-01 06:05Z,2025-09-01 08:05+02:00
0,AF 1449,Barcelona,Air France,Airbus A321,2025-09-01 06:05Z,2025-09-01 08:05+02:00
